# Applications Weighted Grading Automation
<a target="_blank" href="https://colab.research.google.com/github/trendinafrica/student_selection_process_automation/blob/main/weighted_grading.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

(*By: [@mahmoud-elmakki](https://github.com/mahmoud-elmakki)*)

This stage is for calculating the average grade of a response from the grades given by multiple evaluatores. Each reviewer gives a general grade for each response.

Also see [main.ipynb](https://github.com/trendinafrica/student_selection_process_automation/blob/main/main.ipynb), and [weighted_grading_second_round.ipynb](https://github.com/trendinafrica/student_selection_process_automation/blob/main/weighted_grading_second_round.ipynb)..

### **How to use (as a developer):**
Just clone the Github repository and get into the business!\
If you have anaconda and yupyter installed locally you can just clone the repory directly on your machine. Elsewise, you can clone it into Google Colab.
(In either case, if you have any valuable contributions, don't hesitatte to do a Pull Request).

### **How to use (as a reviewer):**
If you are on Github now, open this notebook in Google Colab, or clone the whole repo locally, so you can run the cells. In case of running it in Colab, don't forget to save and download the resulting Excel sheet of the processed responses into a local folder.

In [ ]:
import numpy as np
import pandas as pd

import os

In [ ]:
# Note that you have to download the responses data Excel sheet from Google Drive and put it in the same folder as the code.
# You don't have to do this if you cloned the Github repo (all will be organized in the repo).
 
# TODO: Load data directly from Google Drive.

# Loading students responses data
REVD_STDN_DATA_DIR = './reviewed_responses/Mahmoud_and_Burak/reviewed_dummy_responses_Mahmoud_and_Burak.xlsx'
reviewed_responses = pd.read_excel(REVD_STDN_DATA_DIR)

reviewed_responses['Total Review'] = None

# Just specify folder names - thje code will create the directory
OUTPUT_FOLDER_NAME = "first_grading_round_results"
OUTPUT_DIR = os.path.join(os.getcwd(), OUTPUT_FOLDER_NAME)

if not os.path.exists(OUTPUT_DIR):
    os.mkdir(OUTPUT_DIR)

In [ ]:
# Use this dictionary as a reference for column names.

qs_dict = {i: column for i, column in enumerate(reviewed_responses.columns)}
qs_dict

In [ ]:
# Used indices of the student responses DataFrame

std_idcs = {
    'email_idx' : 1,
    'firstname_idx' : 2,
    'lastname_idx' : 3,
    'nat_idx' : 5,
    'resid_idx' : 6,
    'cv_idx' : 23,
    'ref' : {
        'first_ref_email_idx' : 25,
        'second_ref_email_idx' : 27
          },
    'flag_idx' : 28,
    'notes_idx' : 29,
    'first_recomm_letter_idx' : 30,
    'second_recomm_letter_idx' : 31,
    'reviewer_1_idx' : 32,
    'reviewer_2_idx' : 33,
    'total_review_idx' : 34
    
}

stdn_str_qs = [std_idcs['email_idx'], std_idcs['firstname_idx'], std_idcs['lastname_idx']]

# Carefully specify names of the columns to be processed (CV and recommendation letters).
reviews_idcs = [std_idcs['reviewer_1_idx'], std_idcs['reviewer_2_idx']]
reviews_ws = [1, 1]

WEIGHTED_REVIEWING = False

if not WEIGHTED_REVIEWING:
    reviews_ws = [1, 1]

# Number of students to be selected.
TOP_N = 5

if TOP_N > len(reviewed_responses):
    TOP_N = len(reviewed_responses)

In [ ]:
def column_names_to_indices(df, indices_dict):
    """
    Replaces column names with indices.
    """
    processed_df = df.rename(columns={column: i for i, column in enumerate(indices_dict.values())})

    return processed_df


def indices_to_column_names(df, indices_dict):
    """
    Replaces indices with column names.
    """
    processed_df = df.rename(columns={i: column for i, column in enumerate(indices_dict.values())})

    return processed_df


def calc_review(reviewed_responses, reviews=reviews_idcs, weights=reviews_ws):
    
    for row_index, row in reviewed_responses.iterrows():
        reviews = []
        
        for rev in reviews_idcs:
            review = float(reviewed_responses.iloc[row_index, rev])
            reviews.append(review)
        
        weighted_reviews = [reviews[i] * weights[i] for i in range(len(reviews))]
        total_review = float(round(sum(weighted_reviews) / sum(weights), 2))
        reviewed_responses.iloc[row_index, std_idcs['total_review_idx']] = total_review
            
    return reviewed_responses
 

In [ ]:
def main(reviewed_responses):
    
    reviewed_responses = column_names_to_indices(reviewed_responses, qs_dict)
    reviewed_responses = calc_review(reviewed_responses)
    reviewed_responses_sorted = reviewed_responses.sort_values(by = std_idcs['total_review_idx'], ascending = False)
    
    indices_to_column_names(reviewed_responses, qs_dict).to_excel(OUTPUT_FOLDER_NAME + "/reviewed_responses.xlsx")
    
    top_n_responses = reviewed_responses_sorted.iloc[:TOP_N, :]
    
    indices_to_column_names(top_n_responses, qs_dict).to_excel(OUTPUT_FOLDER_NAME + "/top_n_reviewed_responses.xlsx")
    
    return reviewed_responses, top_n_responses

In [ ]:
reviewed_responses_final, top_n_responses = main(reviewed_responses)

In [ ]:
reviewed_responses_final

In [ ]:
top_n_responses